### Imports

In [1]:
from utils import *
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
from pprint import *
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import math

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


### read data

In [2]:
train = pd.read_csv("../data/train/training_data.csv",delimiter=",",header = None)
train_label = pd.read_csv("../data/train/training_labels.csv",delimiter=",",header = None)
test = pd.read_csv("../data/test/test_data.csv",delimiter=",",header = None)
groups = pd.read_csv("../data/train/training_subjects.csv",delimiter=",",header = None)
t_groups = pd.read_csv("../data/test/test_subjects.csv",delimiter=",",header = None)

### data exploration



In [3]:
pprint(train.shape)
pprint(test.shape)
pprint(train_label.shape)

(6373, 561)
(3926, 561)
(6373, 1)


In [4]:
pprint(groups[0].unique())
pprint(t_groups[0].unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18])
array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])


### data scaling
Since the data is collected by subjects. All features are scaled with in subject.

In [5]:
train_group = 19
test_group = 12

scale_by_group(train,groups,train_group,"train_after_scale.csv")
scale_by_group(test,t_groups,test_group,"test_after_scale.csv")

del groups
del t_groups

train = pd.read_csv("train_after_scale.csv",delimiter=",",header = None)
test = pd.read_csv("test_after_scale.csv",delimiter=",",header = None)

### Light gradient boosting

8 fold corss avlidation

In [6]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)
train_columns = train.columns.values

params = {'num_leaves': 17,
         'min_data_in_leaf': 20,
         'objective':'multiclass',
         'learning_rate': 0.03,
         'max_depth': 30,
         "boosting": "gbdt",
         "feature_fraction": 0.7,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "metric": 'multi_error',
         "lambda_l2": 0.1,
         "verbosity": -1,
         "random_state": 42}

oof = np.zeros(len(train))
y_pred = np.zeros(len(test))
#feature_importance_df = pd.DataFrame()
#run model
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train,train_label.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    
    X_tr, X_val = train.iloc[trn_idx], train.iloc[val_idx]
    y_tr, y_val = train_label.iloc[trn_idx], train_label.iloc[val_idx]

    model = lgb.LGBMClassifier(**params, n_estimators = 1300, n_jobs = -1)
    model.fit(X_tr, 
              y_tr.loc[:,0], 
              eval_set=[(X_tr, y_tr.loc[:,0]), (X_val, y_val.loc[:,0])], 
              eval_metric=['multi_error','multi_logloss'],
              verbose=100, 
              early_stopping_rounds=200)
    oof[val_idx] = model.predict(X_val, num_iteration=model.best_iteration_)
    y_pred += model.predict(test,num_iteration=model.best_iteration_)/n_fold

pprint(accuracy_score(oof,train_label))

fold 0
Training until validation scores don't improve for 200 rounds.
[100]	training's multi_logloss: 0.179975	training's multi_error: 0.0334786	valid_1's multi_logloss: 0.261104	valid_1's multi_error: 0.0423197
[200]	training's multi_logloss: 0.043553	training's multi_error: 0.00209241	valid_1's multi_logloss: 0.20032	valid_1's multi_error: 0.0407524
[300]	training's multi_logloss: 0.0133672	training's multi_error: 0	valid_1's multi_logloss: 0.208406	valid_1's multi_error: 0.0376176
[400]	training's multi_logloss: 0.00456149	training's multi_error: 0	valid_1's multi_logloss: 0.230107	valid_1's multi_error: 0.0360502
Early stopping, best iteration is:
[212]	training's multi_logloss: 0.0374835	training's multi_error: 0.00087184	valid_1's multi_logloss: 0.199453	valid_1's multi_error: 0.039185
fold 1
Training until validation scores don't improve for 200 rounds.
[100]	training's multi_logloss: 0.179891	training's multi_error: 0.0326068	valid_1's multi_logloss: 0.273713	valid_1's multi_er

In [8]:
pd.DataFrame(oof)
df = pd.DataFrame(y_pred)
y_pred = thresholding(y_pred,0.3)
to_submission(y_pred,"redo.csv")

### ensemble model
out of fold hill climb ensemble model

### post-processing